# Confluence + LLM = QA

In [1]:
%pip install pandas --quiet
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


C:\Users\Timur\AppData\Local\Temp\ipykernel_17468\4041452349.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from os import environ
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

confluence_token = environ.get('CONFLUENCE_TOKEN')
hf_token = environ.get('HF_TOKEN')
gigachat_token = environ.get('GIGACHAT_TOKEN')

In [3]:
from atlassian import Confluence

confluence_url = "https://confluence.utmn.ru"
confluence = Confluence(url=confluence_url, token=confluence_token)

## Поиск документа в Confluence через CQL

 * https://atlassian-python-api.readthedocs.io/confluence.html
 * https://developer.atlassian.com/cloud/confluence/rest/v2/api-group-page/#api-pages-id-get
 * https://developer.atlassian.com/server/confluence/advanced-searching-using-cql/
 * https://spacy.io/usage/spacy-101

In [8]:
!python -m spacy download ru_core_news_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [9]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import PyPDFLoader

import spacy
nlp = spacy.load("ru_core_news_sm")
needed_pos = ['NOUN', 'NUM', 'PROPN', 'ADJ', 'VERB', 'X']


def get_cql_query(spaces, question):
    exclude = ' and label != "навигация"' 
    words = [token for token in nlp(question.lower()) if not token.is_stop and
             token.pos_ in needed_pos and len(token.text) > 2]
    if len(words) == 0:
        return ()
    spaces = " or ".join([f"space = {space}" for space in spaces])
    words_with_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                              for word in words])))
    words_without_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                 for word in words if word.pos_ != 'VERB'])))
    words_without_verbs_and_adj = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                         for word in words if word.pos_ not in ['VERB', 'ADJ']])))
    return ("(" + spaces + ") and (" + words_with_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs_and_adj + ")" + exclude)
    
    
def get_document_id(question: str) -> str:
    cql_query = get_cql_query(spaces=["study"], question=question)
    if len(cql_query) == 0:
        return "0"
    results = confluence.cql(cql_query[0], start=0, limit=1)['results']
    if len(results) == 0:
        results = confluence.cql(cql_query[1], start=0, limit=1)['results']
        if len(results) == 0:
            results = confluence.cql(cql_query[2], start=0, limit=1)['results']
            if len(results) == 0:
                return "0"

    return results[0]['content']['id']


def get_document_content_by_id(page_id: str):
    page = confluence.get_page_by_id(page_id, expand='space,body.export_view')
    page_body = page['body']['export_view']['value']
    page_download = page['_links']['base'] + page['_links']['download'] if 'download' in page['_links'].keys() else ''

    try:
        if len(page_body) > 50:
            page_body = page['body']['export_view']['value']
            soup = BeautifulSoup(page_body, 'html.parser')
            page_body_text = soup.get_text(separator=' ')
            content = page_body_text.replace(" \n ", "")
        elif '.pdf' in page_download.lower():
            loader = PyPDFLoader(page_download.split('?')[0])
            content = " ".join([page.page_content for page in loader.load_and_split()])
        else:
            return None
    except:
        return None

    return content


def get_document_content(question: str):
    page_id = get_document_id(question)
    if page_id == "0":
        return None
    return get_document_content_by_id(page_id)


get_document_content("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".\xa0 Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.\xa0 Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.\xa0 Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО!  Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06.09.2023 и будет открыт до конца семестра.\xa0 Для успешной аттестации по дисциплине «Физическая культура:

## Собственный индекс Confluence

### Структура БД

In [10]:
from sqlalchemy import create_engine
engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=True)

In [11]:
from typing import Optional
from pgvector.sqlalchemy import Vector
from sqlalchemy import Text, select, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

class Base(DeclarativeBase):
    pass

class Document(Base):
    __tablename__ = "document"
    id: Mapped[int] = mapped_column(primary_key=True)
    text: Mapped[str] = mapped_column(Text())
    text_lem: Mapped[str] = mapped_column(Text())
    tfidf: Mapped[Optional[Vector]] = mapped_column(Vector(2936))
    doc2vec: Mapped[Optional[Vector]] = mapped_column(Vector(100))
    rubert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert_finetuned: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    

### Выгрузка, предобработка и сохранение документов из пространства

In [12]:
pages = confluence.cql("space = study and label != \"навигация\"", start=0, limit=100)['results']
page_ids = [page['content']['id'] for page in pages if 'content' in page.keys()]
conf_df = pd.DataFrame({"page_id": page_ids})
conf_df["content"] = conf_df["page_id"].apply(get_document_content_by_id)
conf_df

,page_id,content
0,86479083,Сопровождение\nспециализированных\nобразовател...
1,86479057,2 \n \n \n \n1. ОБЩИЕ ПОЛОЖЕНИЯ \n \n1.1. По...
2,86479066,MuHucrepcreo HayKu 14 BbtcuJero o6paaoeaHnR Po...
3,86479079,Об утверждении Регламента \nпроведения промежу...
4,86479078,Об утверждении Р егламента \nпроведения промеж...
5,86479077,MrHucrepcreo HayKn I Bbtctuero o6pasoeaHnR poc...
6,86479076,Об утверждении Регламента \nпроведения промежу...
7,86479075,Об утверждении Регламента \nпроведения промежу...
8,86479074,MrHncrepcreo HayKu 14 Bbtcuero o6paroeaHnn poc...
9,86479073,MrHucrepcreo HayKu t4 Bbtcuero o6pa:oeaHnR Poc...


In [13]:
trash_id = ["86479082", "86479093", "86479069", "86479070", "86479071", "86479072", "86479073", "86479074", "86479077", "86479066"]
conf_df["page_id"] = conf_df["page_id"].apply(lambda x: None if x in trash_id else x)
conf_df = conf_df.dropna().reset_index(drop=True)
conf_df.head(5)

,page_id,content
0,86479083,Сопровождение\nспециализированных\nобразовател...
1,86479057,2 \n \n \n \n1. ОБЩИЕ ПОЛОЖЕНИЯ \n \n1.1. По...
2,86479079,Об утверждении Регламента \nпроведения промежу...
3,86479078,Об утверждении Р егламента \nпроведения промеж...
4,86479076,Об утверждении Регламента \nпроведения промежу...


In [14]:
with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    session.commit()
Base.metadata.create_all(engine)

2024-02-13 16:16:35,608 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-02-13 16:16:35,610 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 16:16:35,614 INFO sqlalchemy.engine.Engine select current_schema()
2024-02-13 16:16:35,615 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 16:16:35,619 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-02-13 16:16:35,619 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-13 16:16:35,623 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:16:35,624 INFO sqlalchemy.engine.Engine CREATE EXTENSION IF NOT EXISTS vector
2024-02-13 16:16:35,625 INFO sqlalchemy.engine.Engine [generated in 0.00074s] {}
2024-02-13 16:16:35,629 INFO sqlalchemy.engine.Engine COMMIT
2024-02-13 16:16:35,631 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:16:35,635 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid =

In [15]:
def lower_stopword_lemmatize(text):
    return " ".join([token.lemma_ for token in nlp(str(text).lower()) if not token.is_stop and token.pos_ != 'PUNCT'])

In [16]:
with Session(engine) as session:
    for index, row in conf_df.iterrows():
        doc = Document(
            text=row["content"], 
            text_lem=lower_stopword_lemmatize(row["content"])
        )
        session.add(doc)
    session.commit()   

2024-02-13 16:17:03,027 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:17:03,034 INFO sqlalchemy.engine.Engine INSERT INTO document (text, text_lem, tfidf, doc2vec, rubert, rusbert, rusbert_finetuned) SELECT p0::TEXT, p1::TEXT, p2::VECTOR(2936), p3::VECTOR(100), p4::VECTOR(312), p5::VECTOR(312), p6::VECTOR(312) FROM (VALUES (%(text__0)s, %(text_lem__0)s, %(tf ... 4841 characters truncated ... 1, p2, p3, p4, p5, p6, sen_counter) ORDER BY sen_counter RETURNING document.id, document.id AS id__1
2024-02-13 16:17:03,034 INFO sqlalchemy.engine.Engine [generated in 0.00029s (insertmanyvalues) 1/1 (ordered)] {'rusbert__0': None, 'tfidf__0': None, 'rubert__0': None, 'text__0': 'Сопровождение\nспециализированных\nобразовательных треков Спорт.\nпрограмми -\nрование\nТреки\nКапитаныШкола \nестественных\nнаукИнтеграция Нетология ... (3774 characters truncated) ... я 46 студентовСпорт.\nпрограмми -\nрование\nЧисленность треков\n57 студентовКапитаны\n80 студентовШЕН\n146 студентовИнтегр

### Эмбеддинги

In [18]:
with Session(engine) as session:
   db_documents_lem = pd.Series([doc.text_lem for doc in session.scalars(select(Document).order_by(Document.id)).all()])
db_documents_lem.head()

2024-02-13 16:18:41,011 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:18:41,013 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.id
2024-02-13 16:18:41,014 INFO sqlalchemy.engine.Engine [generated in 0.00144s] {}
2024-02-13 16:18:41,043 INFO sqlalchemy.engine.Engine ROLLBACK


0    сопровождение \n специализированный \n образов...
1    2 \n \n \n \n 1 общий положение  \n \n 1.1   п...
2    утверждение регламент \n проведение промежуточ...
3    утверждение егламента \n проведение промежуточ...
4    утверждение регламент \n проведение промежуточ...
dtype: object

#### TFIDF

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidfX = tfidf_vectorizer.fit_transform(db_documents_lem)
len(tfidf_vectorizer.transform([db_documents_lem[0]]).toarray()[0])

2936

In [20]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.tfidf = tfidf_vectorizer.transform([doc.text_lem]).toarray()[0]
      session.add(doc)
      session.flush()
   session.commit()

2024-02-13 16:19:01,320 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:19:01,321 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.id
2024-02-13 16:19:01,323 INFO sqlalchemy.engine.Engine [cached since 20.31s ago] {}
2024-02-13 16:19:01,355 INFO sqlalchemy.engine.Engine UPDATE document SET tfidf=%(tfidf)s WHERE document.id = %(document_id)s
2024-02-13 16:19:01,357 INFO sqlalchemy.engine.Engine [generated in 0.00234s] {'tfidf': '[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04363435541903708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02442582110088 ... (13740 characters truncated) ... 772566793131985,0.024425821100884624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]', 'document_id': 1}
2024-02-13 16:19:01,365 INFO sqlal

In [21]:
def answer_tfidf(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.tfidf.cosine_distance(
                            tfidf_vectorizer.transform([lower_stopword_lemmatize(question)]).toarray()[0]
                            )).limit(1)).first().text
        
answer_tfidf("Как поменять физкультуру?")

2024-02-13 16:19:07,775 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:19:07,777 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.tfidf <=> %(tfidf_1)s 
 LIMIT %(param_1)s
2024-02-13 16:19:07,779 INFO sqlalchemy.engine.Engine [generated in 0.00215s] {'tfidf_1': '[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ... (11477 characters truncated) ... ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]', 'param_1': 1}
2024-02-13 16:19:07,786 INFO sqlalchemy.engine.Engine ROLLBACK


'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра. Элективы 2023.PDF'

#### Doc2vec

In [29]:
%pip install gensim --quiet

Note: you may need to restart the kernel to use updated packages.


In [23]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([text.split() for text in db_documents_lem])]
doc2vec_model = Doc2Vec(documents, vector_size=100, window=3, min_count=1, workers=4)

len(doc2vec_model.infer_vector("мама мыла раму".split()))

100

In [24]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.doc2vec = doc2vec_model.infer_vector(doc.text_lem.split())
      session.add(doc)
      session.flush()
   session.commit()

2024-02-13 16:19:41,283 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:19:41,284 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.id
2024-02-13 16:19:41,285 INFO sqlalchemy.engine.Engine [cached since 60.27s ago] {}
2024-02-13 16:19:41,337 INFO sqlalchemy.engine.Engine UPDATE document SET doc2vec=%(doc2vec)s WHERE document.id = %(document_id)s
2024-02-13 16:19:41,340 INFO sqlalchemy.engine.Engine [generated in 0.00185s] {'doc2vec': '[-0.16956333816051483,-0.8514053821563721,-1.3372600078582764,1.050919532775879,0.11938895285129547,-0.028881030157208443,0.8220304250717163,0.753205 ... (1657 characters truncated) ... 00190591812134,-0.9721548557281494,0.17025057971477509,0.9183670282363892,0.6791316866874695,-1.0436192750930786,2.353215456008911,1.426676869392395]', 'document_id': 1}
2024-02-13 16:19:41,357 INFO 

In [25]:
def answer_doc2vec(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.doc2vec.cosine_distance(
                           doc2vec_model.infer_vector(lower_stopword_lemmatize(question).split())
                            )).limit(1)).first().text
        
answer_doc2vec("Как поменять физкультуру?")

2024-02-13 16:19:49,555 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:19:49,557 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.doc2vec <=> %(doc2vec_1)s 
 LIMIT %(param_1)s
2024-02-13 16:19:49,558 INFO sqlalchemy.engine.Engine [generated in 0.00127s] {'doc2vec_1': '[0.012717717327177525,0.009471967816352844,-0.026494335383176804,0.004672396928071976,-0.005535815376788378,0.008504985831677914,0.01643168367445469, ... (1837 characters truncated) ... ,-0.010590136051177979,0.026128411293029785,0.007570881862193346,0.01730772666633129,-0.015697073191404343,0.035792045295238495,0.015028919093310833]', 'param_1': 1}
2024-02-13 16:19:49,564 INFO sqlalchemy.engine.Engine ROLLBACK


'Для рассмотрения возможности продления академического отпуска необходимо подать два заявления:  на выход из отпуска,  на предоставление нового отпуска. Вы можете подать заявления лично, обратившись в Единый деканат (ул Семакова, 18, холл 3 этаж) с паспортом или студенческим билетом, либо направить подписанный своей рукой бланк на e-mail:  ed@utmn.ru \xa0 Бланки заявления:  https://www.utmn.ru/o-tyumgu/organizatsionnaya-skhema-tyumgu/edinyy-dekanat/blanki-zayavleniy/otpusk/ .\xa0'

#### RuBERT-Tiny

In [26]:
import torch
from transformers import AutoTokenizer, AutoModel
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
rubert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
rubert_model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

len(embed_bert_cls("мама мыла раму", rubert_model, rubert_tokenizer))

d:\git\virtassist\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


312

In [27]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.rubert = embed_bert_cls(doc.text, rubert_model, rubert_tokenizer)
      session.add(doc)
      session.flush()
   session.commit()

2024-02-13 16:20:00,469 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:20:00,471 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.id
2024-02-13 16:20:00,471 INFO sqlalchemy.engine.Engine [cached since 79.46s ago] {}
2024-02-13 16:20:00,557 INFO sqlalchemy.engine.Engine UPDATE document SET rubert=%(rubert)s WHERE document.id = %(document_id)s
2024-02-13 16:20:00,560 INFO sqlalchemy.engine.Engine [generated in 0.00271s] {'rubert': '[0.07620523124933243,0.04829400032758713,0.04743478074669838,-0.031931884586811066,0.030682871118187904,-0.011257819831371307,-0.04609350115060806,-0 ... (6267 characters truncated) ... 6,0.004650117829442024,0.016865519806742668,0.016757512465119362,0.05118463188409805,0.04259665682911873,-0.010972267016768456,-0.009127729572355747]', 'document_id': 1}
2024-02-13 16:20:00,597 INFO sql

In [28]:
def answer_rubert(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.rubert.cosine_distance(
                            embed_bert_cls(question, rubert_model, rubert_tokenizer)
                            )).limit(1)).first().text
        
answer_rubert("Как поменять физкультуру?")

2024-02-13 16:20:03,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:20:03,934 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.rubert <=> %(rubert_1)s 
 LIMIT %(param_1)s
2024-02-13 16:20:03,936 INFO sqlalchemy.engine.Engine [generated in 0.00177s] {'rubert_1': '[0.03047611005604267,-0.0251536276191473,0.011420643888413906,-0.07160034030675888,0.04369772598147392,-0.034605640918016434,0.006868361029773951,0.0 ... (6218 characters truncated) ... -0.045723091810941696,0.03559586778283119,-0.03248249739408493,-0.029997270554304123,0.049736715853214264,0.023025432601571083,-0.016017327085137367]', 'param_1': 1}
2024-02-13 16:20:03,943 INFO sqlalchemy.engine.Engine ROLLBACK


'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра. Элективы 2023.PDF'

#### RuSBERT-Tiny

In [29]:
from sentence_transformers import SentenceTransformer

rusbert_model = SentenceTransformer('cointegrated/rubert-tiny2')
len(rusbert_model.encode("мама мыла раму"))

312

In [30]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.rusbert = rusbert_model.encode(doc.text)
      session.add(doc)
      session.flush()
   session.commit()

2024-02-13 16:20:16,624 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:20:16,625 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.id
2024-02-13 16:20:16,626 INFO sqlalchemy.engine.Engine [cached since 95.61s ago] {}
2024-02-13 16:20:16,838 INFO sqlalchemy.engine.Engine UPDATE document SET rusbert=%(rusbert)s WHERE document.id = %(document_id)s
2024-02-13 16:20:16,839 INFO sqlalchemy.engine.Engine [generated in 0.00152s] {'rusbert': '[0.06631461530923843,0.03493586555123329,0.06720124930143356,0.0011828569695353508,-0.020561115816235542,-0.013000190258026123,-0.012056092731654644, ... (6218 characters truncated) ... .02145107090473175,-0.0020276980940252542,0.022892959415912628,0.0029329818207770586,0.04563396796584129,0.0013529256684705615,-0.002223630901426077]', 'document_id': 1}
2024-02-13 16:20:17,028 INFO 

In [31]:
def answer_rusbert(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.rusbert.cosine_distance(
                            rusbert_model.encode(question)
                            )).limit(1)).first().text
        
answer_rubert("Как поменять физкультуру?")

2024-02-13 16:20:21,661 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:20:21,663 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.rubert <=> %(rubert_1)s 
 LIMIT %(param_1)s
2024-02-13 16:20:21,664 INFO sqlalchemy.engine.Engine [cached since 17.73s ago] {'rubert_1': '[0.03047611005604267,-0.0251536276191473,0.011420643888413906,-0.07160034030675888,0.04369772598147392,-0.034605640918016434,0.006868361029773951,0.0 ... (6218 characters truncated) ... -0.045723091810941696,0.03559586778283119,-0.03248249739408493,-0.029997270554304123,0.049736715853214264,0.023025432601571083,-0.016017327085137367]', 'param_1': 1}
2024-02-13 16:20:21,669 INFO sqlalchemy.engine.Engine ROLLBACK


'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра. Элективы 2023.PDF'

## Выбор нужного фрагмента через векторный индекс

### Тестовая выборка с вопроосами

In [59]:
questions = pd.read_csv("questions.csv", index_col=0)
questions

,question,benchmark
0,где почитать отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи..."
1,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи..."
2,как выбрать электив?,"Прежде, чем выбрать элективы, рекомендуем почи..."
3,"что делать, если военкомат просит справку?",Вам необходимо обратиться в Отдел мобилизацион...
4,где посмотреть отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи..."
5,как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...
6,сколько баллов нужно на какую оценку?,Документ подписан простой электронной подписью...
7,как поменять физкультуру?,Выбор спортивных секций по Физической культуре...
8,как подать заявления на академ?,Заявление на академический отпуск подается чер...
9,что делать при потере студенческого билета?,Для восстановления студенческого билета Вам ...


In [60]:
questions["CQL"] = questions["question"].apply(get_document_content)
questions["tfidf"] = questions["question"].apply(answer_tfidf)
questions["doc2vec"] = questions["question"].apply(answer_doc2vec)
questions["rubert"] = questions["question"].apply(answer_rubert)
questions["rusbert"] = questions["question"].apply(answer_rusbert)
questions

2024-02-13 16:28:32,330 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:28:32,330 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.tfidf <=> %(tfidf_1)s 
 LIMIT %(param_1)s
2024-02-13 16:28:32,330 INFO sqlalchemy.engine.Engine [cached since 564.6s ago] {'tfidf_1': '[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ... (11492 characters truncated) ... 0,0.0,0.0,0.0,0.5359932542057623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]', 'param_1': 1}
2024-02-13 16:28:32,341 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-13 16:28:32,351 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:28:32,354 INFO sqlalchemy.engine.Engine SELECT document.

,question,benchmark,CQL,tfidf,doc2vec,rubert,rusbert
0,где почитать отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Для рассмотрения возможности продления академи...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
1,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","По вопросам, касающимся общежития, обратитесь ...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
2,как выбрать электив?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
3,"что делать, если военкомат просит справку?",Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...,Для оформления академической справки (справки ...,Пакет документов для восстановления вам необхо...,"При восстановлении на договорное место, после ...","При восстановлении на договорное место, после ..."
4,где посмотреть отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
5,как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...,Заявления о переводе принимаются два раза в го...,"Восстановление лица , ранее обучавшегося за сч...","1. Подать заявление о переводе можно лично, об...",Для рассмотрения возможности продления академи...,Подать заявление на отчисление переводом можно...
6,сколько баллов нужно на какую оценку?,Документ подписан простой электронной подписью...,Документ подписан простой электронной подписью...,Об утверждении Регламента \nпроведения промежу...,Об утверждении Регламента \nпроведения промежу...,"По вопросу получения справки о доходах, размер...",Об утверждении Регламента \nпроведения промежу...
7,как поменять физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...,"Прежде, чем выбрать элективы, рекомендуем почи...","По вопросам оплаты обучения, суммы задолженнос...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...
8,как подать заявления на академ?,Заявление на академический отпуск подается чер...,Заявление на академический отпуск подается чер...,Для рассмотрения возможности продления академи...,Заявление на академический отпуск подается чер...,"Прежде, чем выбрать элективы, рекомендуем почи...",Заявление на выход из отпуска подается не позд...
9,что делать при потере студенческого билета?,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам ...,Для рассмотрения возможности продления академи...,Для восстановления студенческого билета Вам ...


### Выбор лучшего алгоритма: BERTScore

In [50]:
%pip install bert_score --quiet

Note: you may need to restart the kernel to use updated packages.


In [34]:
from bert_score import BERTScorer

scorer = BERTScorer(lang="ru")

In [35]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.benchmark))
f"benchmark. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'benchmark. Precision: 1.000 Recall: 1.000 F1 score: 1.000'

In [36]:
sum(questions.benchmark == questions.benchmark) / len(questions.benchmark)

1.0

In [37]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.CQL.apply(lambda x: "" if x is None else x)))
f"CQL. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'CQL. Precision: 0.724 Recall: 0.720 F1 score: 0.722'

In [38]:
sum(questions.CQL.apply(lambda x: "" if x is None else x) == questions.benchmark) / len(questions.benchmark)

0.6538461538461539

In [39]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.tfidf))
f"TFIDF. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'TFIDF. Precision: 0.829 Recall: 0.853 F1 score: 0.840'

In [40]:
sum(questions.tfidf == questions.benchmark) / len(questions.benchmark)

0.5384615384615384

In [41]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.doc2vec))
f"doc2vec. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'doc2vec. Precision: 0.628 Recall: 0.615 F1 score: 0.620'

In [42]:
sum(questions.doc2vec == questions.benchmark) / len(questions.benchmark)

0.038461538461538464

In [43]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.rubert))
f"rubert. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'rubert. Precision: 0.701 Recall: 0.719 F1 score: 0.709'

In [44]:
sum(questions.rubert == questions.benchmark) / len(questions.benchmark)

0.23076923076923078

In [45]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.rusbert))
f"rusbert. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'rusbert. Precision: 0.720 Recall: 0.736 F1 score: 0.727'

In [46]:
sum(questions.rusbert == questions.benchmark) / len(questions.benchmark)

0.3076923076923077

## SBERT Fine Tuning

 * https://www.sbert.net/docs/training/overview.html
 * https://huggingface.co/blog/how-to-train-sentence-transformers

### Генерация обучающей выборки через GigaChat

In [47]:
with Session(engine) as session:
   db_documents = [doc.text for doc in session.scalars(select(Document).order_by(Document.id)).all() if len(doc.text) < 3000]
len(db_documents)

2024-02-13 16:21:43,438 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:21:43,439 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.id
2024-02-13 16:21:43,440 INFO sqlalchemy.engine.Engine [cached since 182.4s ago] {}
2024-02-13 16:21:43,502 INFO sqlalchemy.engine.Engine ROLLBACK


28

In [48]:
from langchain.prompts import PromptTemplate

prompt_template = """
Сделай глубокий вдох и действуй как студент. На какие 5 вопросов ты можешь получить ответы из документа в тройных кавычках? Используй разговорный стиль речи и студенческую лексику.

\"\"\"
{content}
\"\"\"

Вопросы:
"""

prompt = PromptTemplate.from_template(prompt_template)

In [49]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga
giga_chain.invoke({"content": db_documents[0]}).strip().split("\n")

['1. Когда начинается выбор спортивных секций по Физической культуре?',
 '2. Какие ограничения существуют при записи на спортивные секции?',
 '3. Как проверить наличие конфликтов в расписании спортивных секций и своем расписании?',
 '4. Как изменить выбор спортивных секций?',
 '5. Что произойдет, если студент пропустит два занятия подряд?']

In [50]:
gigachat_docs = []
for doc in db_documents:
    query = {"content": doc}
    giga_questions = giga_chain.invoke(query).strip().split("\n")
    for q in giga_questions:
        gigachat_docs.append({
            "question": q[3:],
            "document": doc
        })
    print(giga_questions)
gigachat_docs = pd.DataFrame(gigachat_docs)
gigachat_docs

['1. Когда начинается выбор спортивных секций по Физической культуре?', '2. Какие ограничения существуют при записи на спортивные секции?', '3. Как проверить наличие конфликтов в расписании спортивных секций и своем расписании?', '4. Как изменить выбор спортивных секций?', '5. Что произойдет, если студент пропустит два занятия подряд?']
['1. Что такое "Отзывус"?', '2. Как можно оставить отзыв на "Отзывусе"?', '3. Где можно найти информацию о том, как поменять электив?', '4. Какая информация отправляется на корпоративную почту в первую учебную неделю семестра?', '5. Какие элективы доступны в 2023 году?']
['1. Какие способы оплаты обучения доступны?', '2. Как связаться с отделом платных образовательных услуг?', '3. Где находится отдел платных образовательных услуг?', '4. Как получить документы для оплаты за обучение материнским капиталом?', '5. Каковы сроки оплаты по договору для студентов разных форм обучения?']
['1. Где можно оформить справку о подтверждении обучения?', '2. Какие терми

,question,document
0,Когда начинается выбор спортивных секций по Фи...,Выбор спортивных секций по Физической культуре...
1,Какие ограничения существуют при записи на спо...,Выбор спортивных секций по Физической культуре...
2,Как проверить наличие конфликтов в расписании ...,Выбор спортивных секций по Физической культуре...
3,Как изменить выбор спортивных секций?,Выбор спортивных секций по Физической культуре...
4,"Что произойдет, если студент пропустит два зан...",Выбор спортивных секций по Физической культуре...
...,...,...
153,Как подать заявление на отчисление переводом?,Подать заявление на отчисление переводом можно...
154,Где находится Единый деканат?,Подать заявление на отчисление переводом можно...
155,Какие документы нужно приложить к заявлению на...,Подать заявление на отчисление переводом можно...
156,Сколько времени занимает подготовка приказа об...,Подать заявление на отчисление переводом можно...


In [51]:
gigachat_docs.to_csv("gigachat_docs.csv")

### Тонкая настройка

In [52]:
gigachat_docs = pd.read_csv("gigachat_docs.csv", index_col=0).reset_index(drop=True)
gigachat_docs

,question,document
0,Когда начинается выбор спортивных секций по Фи...,Выбор спортивных секций по Физической культуре...
1,Какие ограничения существуют при записи на спо...,Выбор спортивных секций по Физической культуре...
2,Как проверить наличие конфликтов в расписании ...,Выбор спортивных секций по Физической культуре...
3,Как изменить выбор спортивных секций?,Выбор спортивных секций по Физической культуре...
4,"Что произойдет, если студент пропустит два зан...",Выбор спортивных секций по Физической культуре...
...,...,...
135,Как подать заявление на отчисление переводом?,Подать заявление на отчисление переводом можно...
136,Где находится Единый деканат?,Подать заявление на отчисление переводом можно...
137,Какие документы нужно приложить к заявлению на...,Подать заявление на отчисление переводом можно...
138,Сколько времени занимает подготовка приказа об...,Подать заявление на отчисление переводом можно...


In [53]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

train_set = []
for index, row in gigachat_docs.iterrows():
    train_set.append(InputExample(texts=[row['question'], row['document']]))
    
finetuned_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cuda")

train_dataloader = DataLoader(train_set, shuffle=True, batch_size=16)
train_loss = losses.MegaBatchMarginLoss(finetuned_model)

finetuned_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=100)


Epoch: 100%|██████████| 2/2 [00:03<00:00,  1.64s/it]


In [54]:
finetuned_model.save("saved_models/rubert-tiny2-wikiutmn-gigachat-qa")

### Индексация и проверка

In [55]:
with Session(engine) as session:
   documents = session.scalars(select(Document).order_by(Document.id)).all()
   for doc in documents:
      doc.rusbert_finetuned = finetuned_model.encode(doc.text)
      session.add(doc)
      session.flush()
   session.commit()

2024-02-13 16:25:45,111 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:25:45,113 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.id
2024-02-13 16:25:45,114 INFO sqlalchemy.engine.Engine [cached since 424.1s ago] {}
2024-02-13 16:25:45,324 INFO sqlalchemy.engine.Engine UPDATE document SET rusbert_finetuned=%(rusbert_finetuned)s WHERE document.id = %(document_id)s
2024-02-13 16:25:45,325 INFO sqlalchemy.engine.Engine [generated in 0.00144s] {'rusbert_finetuned': '[0.06429866701364517,0.033652957528829575,0.06722468137741089,0.0010618041269481182,-0.021043192595243454,-0.012169316411018372,-0.012706323526799679 ... (6214 characters truncated) ... .02175769954919815,-0.0013543440727517009,0.02315693534910679,0.002602623077109456,0.04555196687579155,-0.0005775493336841464,-0.0007885639788582921]', 'document_id': 1}

In [56]:
def answer_rusbert_finetuned(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.rusbert_finetuned.cosine_distance(
                            finetuned_model.encode(question)
                            )).limit(1)).first().text
        
answer_rubert("Как поменять физкультуру?")

2024-02-13 16:26:13,185 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:26:13,185 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.rubert <=> %(rubert_1)s 
 LIMIT %(param_1)s
2024-02-13 16:26:13,190 INFO sqlalchemy.engine.Engine [cached since 369.3s ago] {'rubert_1': '[0.03047611005604267,-0.0251536276191473,0.011420643888413906,-0.07160034030675888,0.04369772598147392,-0.034605640918016434,0.006868361029773951,0.0 ... (6218 characters truncated) ... -0.045723091810941696,0.03559586778283119,-0.03248249739408493,-0.029997270554304123,0.049736715853214264,0.023025432601571083,-0.016017327085137367]', 'param_1': 1}
2024-02-13 16:26:13,196 INFO sqlalchemy.engine.Engine ROLLBACK


'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра. Элективы 2023.PDF'

In [61]:
questions["rusbert_finetuned"] = questions["question"].apply(answer_rusbert_finetuned)
questions

2024-02-13 16:28:43,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:28:43,130 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.rusbert_finetuned <=> %(rusbert_finetuned_1)s 
 LIMIT %(param_1)s
2024-02-13 16:28:43,134 INFO sqlalchemy.engine.Engine [generated in 0.00117s] {'rusbert_finetuned_1': '[-0.005713378079235554,0.005700490903109312,0.023801613599061966,-0.07757668197154999,-0.03531023859977722,-0.01319127343595028,0.0025229749735444784 ... (6232 characters truncated) ... -0.08214303851127625,0.04335873946547508,-0.03886939957737923,-0.025471799075603485,-0.04935416206717491,-0.002821881789714098,-0.008325671777129173]', 'param_1': 1}
2024-02-13 16:28:43,141 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-13 16:28:43,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:28:43,154 INFO sqlalche

2024-02-13 16:28:43,191 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:28:43,194 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.text_lem, document.tfidf, document.doc2vec, document.rubert, document.rusbert, document.rusbert_finetuned 
FROM document ORDER BY document.rusbert_finetuned <=> %(rusbert_finetuned_1)s 
 LIMIT %(param_1)s
2024-02-13 16:28:43,194 INFO sqlalchemy.engine.Engine [cached since 0.06095s ago] {'rusbert_finetuned_1': '[0.0776311382651329,-0.05627124756574631,-0.05980581045150757,-0.059835728257894516,-0.020278990268707275,-0.03945579752326012,-0.0020326843950897455 ... (6215 characters truncated) ... ,-0.05532272905111313,0.04328654706478119,-0.011766109615564346,0.06510694324970245,0.04506867378950119,-0.0022629741579294205,-0.009241187945008278]', 'param_1': 1}
2024-02-13 16:28:43,201 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-13 16:28:43,213 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-13 16:28:43,215 INFO sqla

,question,benchmark,CQL,tfidf,doc2vec,rubert,rusbert,rusbert_finetuned
0,где почитать отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Для рассмотрения возможности продления академи...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
1,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","По вопросам, касающимся общежития, обратитесь ...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
2,как выбрать электив?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
3,"что делать, если военкомат просит справку?",Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...,Для оформления академической справки (справки ...,Пакет документов для восстановления вам необхо...,"При восстановлении на договорное место, после ...","При восстановлении на договорное место, после ...","При восстановлении на договорное место, после ..."
4,где посмотреть отзывы на элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
5,как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...,Заявления о переводе принимаются два раза в го...,"Восстановление лица , ранее обучавшегося за сч...","1. Подать заявление о переводе можно лично, об...",Для рассмотрения возможности продления академи...,Подать заявление на отчисление переводом можно...,Подать заявление на отчисление переводом можно...
6,сколько баллов нужно на какую оценку?,Документ подписан простой электронной подписью...,Документ подписан простой электронной подписью...,Об утверждении Регламента \nпроведения промежу...,Об утверждении Регламента \nпроведения промежу...,"По вопросу получения справки о доходах, размер...",Об утверждении Регламента \nпроведения промежу...,Об утверждении Регламента \nпроведения промежу...
7,как поменять физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...,"Прежде, чем выбрать элективы, рекомендуем почи...","По вопросам оплаты обучения, суммы задолженнос...","Прежде, чем выбрать элективы, рекомендуем почи...",Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...
8,как подать заявления на академ?,Заявление на академический отпуск подается чер...,Заявление на академический отпуск подается чер...,Для рассмотрения возможности продления академи...,Заявление на академический отпуск подается чер...,"Прежде, чем выбрать элективы, рекомендуем почи...",Заявление на выход из отпуска подается не позд...,Заявление на выход из отпуска подается не позд...
9,что делать при потере студенческого билета?,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам ...,Для рассмотрения возможности продления академи...,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам ...


In [62]:
P, R, F1 = scorer.score(list(questions.rusbert_finetuned), list(questions.benchmark))
f"benchmark. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'benchmark. Precision: 0.740 Recall: 0.718 F1 score: 0.728'

In [63]:
sum(questions.rusbert_finetuned == questions.benchmark) / len(questions.benchmark)

0.3076923076923077

## Большие языковые модели

https://python.langchain.com/docs/use_cases/question_answering/

In [247]:
from langchain.prompts import PromptTemplate

prompt_template = """
Используй следующий текст в тройных кавычках, чтобы кратко ответить на вопрос студента в конце. 
Не изменяй и не убирай ссылки, адреса и телефоны. Если ты не можешь найти ответ, напиши, что ответ не найден.
Ответ не должен превышать 100 слов.

\"\"\"
{context}
\"\"\"

Вопрос: {question}
"""

prompt = PromptTemplate.from_template(prompt_template)

### GigaChat

In [248]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga

In [271]:
# question = questions.question[89]
question = "Самый крутой сервис по выбору авиабилетов?"
print(question, end="\n\n")
document = answer_rusbert(question)
print(document)

Самый крутой сервис по выбору авиабилетов?

2024-02-11 21:06:44,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 21:06:44,386 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.rusbert <=> %(rusbert_1)s 
 LIMIT %(param_1)s
2024-02-11 21:06:44,386 INFO sqlalchemy.engine.Engine [cached since 872.8s ago] {'rusbert_1': '[0.005533029790967703,0.013334708288311958,-0.03015470691025257,-0.1701069474220276,-0.043242618441581726,0.04188287630677223,-0.031845852732658386,0 ... (6246 characters truncated) ... 24,-0.011155599728226662,0.009449596516788006,-0.02684507891535759,0.05969000235199928,0.012244931422173977,0.05982932820916176,-0.09153693169355392]', 'param_1': 1}
2024-02-11 21:06:44,393 INFO sqlalchemy.engine.Engine ROLLBACK
Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём 

In [272]:
query = {"context": document,
        "question": question}
print(giga_chain.invoke(query).strip())
print()
print(query)

Ответ не найден.

{'context': 'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра. Элективы 2023.PDF', 'question': 'Самый крутой сервис по выбору авиабилетов?'}


### Локальные Text2Text

In [25]:
import torch
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

device = torch.cuda.current_device() if torch.cuda.is_available() and torch.cuda.mem_get_info()[0] >= 2*1024**3 else -1
device

d:\git\virtassist\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0

In [26]:
# FOR SAVING MODEL ONLY!
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# model_name = "IlyaGusev/fred_t5_ru_turbo_alpaca"
# model_name = "ai-forever/FRED-T5-large"
# model_name = "ai-forever/FRED-T5-1.7B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# pipe = pipeline(
#     "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500
# )
# pipe.save_pretrained("saved_models/FRED-T5-1.7B")

tokenizer_config.json: 100%|██████████| 20.2k/20.2k [00:00<00:00, 15.1MB/s]
d:\git\virtassist\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timur\.cache\huggingface\hub\models--ai-forever--FRED-T5-1.7B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.json: 100%|██████████| 1.71M/1.71M 

In [27]:
saved_pipeline = pipeline("text2text-generation", "saved_models/FRED-T5-1.7B", device=device, max_new_tokens=10000)
hf_model = HuggingFacePipeline(pipeline=saved_pipeline).bind(stop=["\n\n"])
gpu_chain = prompt | hf_model

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.51s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
content = found_doc['page_content']
query = {"context": content,
        "question": question}
print(gpu_chain.invoke(query).strip())
print()
print(query)

Вопрос: Как мне узнать, что я записан на модуль?
1.Перейти в раздел ИОТ –Выбор модулей
2.В случае, если окно выбора открыто только на просмотр, 
дождаться даты и времени открытия окна на запись
3.Выбрать интересующую дисциплину
4.Осуществить запись в каждый из типов учебных команд, 
доступных для выбора (например, лекционные команды или команды 
по практике)
5.Нажать кнопку Выбрать
6.В случае успешной записи на дисциплину в правом верхнем 
углу появляется всплывающее окно об оформленном выборе
7.Рядом с выбранной дисциплиной появляется значок
 |Критерий завершения уже выполнен, выбрано необходимое количество 
дисциплин или на необходимое количество единиц ценности
8.Рядом с выбранной дисциплиной появляется значок
 |Недостаточно свободных мест в учебной команде
9.Во всплывающем окне выбрать вариант Да, отписаться
10.Отмена записи на дисциплину
11.Просмотр описания модуля
12.Просмотр расписания модуля и всех выбранных модулей
13.Просмотр состава команды
14.Отмена записи на дисциплину
15.

#### Гигиена

In [18]:
del gpu_chain
del hf_model
del saved_pipeline
torch.cuda.empty_cache()